In [194]:
import pandas as pd
# Import required libraries and dependencies
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


import numpy as np

# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [195]:
attributes_df = pd.read_csv("../Resources/songAttributes_1999-2019.csv", encoding='unicode_escape')


billboard_df = pd.read_csv("../Resources/billboardHot100_1999-2019.csv", encoding='unicode_escape')

## Preparing files


In [196]:
# billboard_df.drop(columns=["Unnamed: 0"], inplace=True)
# attributes_df.drop(columns=["Unnamed: 0"], inplace=True)
billboard_df.rename(columns={"Artists":"Artist"}, inplace=True)

In [197]:
billboard_df["Artist"] = billboard_df["Artist"].str.lower()
billboard_df["Name"] = billboard_df["Name"].str.lower()

attributes_df["Artist"] = attributes_df["Artist"].str.lower()
attributes_df["Name"] = attributes_df["Name"].str.lower()


## Merging data

In [198]:
billboard_attributes_df = billboard_df.merge(attributes_df, on=["Name", "Artist"], how ="inner")

In [199]:
pd.set_option('display.max_columns', None)

In [200]:
df_01 = billboard_attributes_df.drop_duplicates(subset=("Name", "Artist", "Album"), keep="first")

In [201]:
df_01.reset_index(inplace=True)
df_01.drop(columns=["index"], inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_21740\998586110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_01.drop(columns=["index"], inplace=True)


In [202]:
df_01

,Unnamed: 0,Artist,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features,X,Acousticness,Album,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,TimeSignature,Valence
0,6,jonas brothers,sucker,6,1.000000,17.000000,2019-07-06,"March 1, 2019","Alternative Pop,Boy Band,Teen Pop,Pop-Rock,Pop","Ryan tedder, Louis bell, Frank dukes, Nick jon...",Sucker \nWe go together\nBetter than birds of ...,NaN,12,0.042700,Happiness Begins,0.842000,181027,0.734000,False,0.000000,0.106000,-5.065000,0,81,0.058800,137.958000,4,0.952000
1,13,taylor swift,you need to calm down,13,2.000000,2.000000,2019-07-06,"June 14, 2019","Synth-Pop,LGBTQ+,Pop","Joel little, Taylor swift",You Need To Calm Down \nYou are somebody that ...,NaN,113,0.009290,Lover,0.771000,171360,0.671000,False,0.000000,0.063700,-5.617000,1,89,0.055300,85.026000,4,0.714000
2,24,panic! at the disco,"hey look ma, i made it",24,24.000000,11.000000,2019-07-06,"June 22, 2018","Pop-Rock,Jazz Fusion,Alternative,Alternative P...","Jake sinclair, Michael angelakos, Dillon franc...","Hey Look Ma, I Made It \nAll my life, been hus...",NaN,80,0.013700,Pray for the Wicked,0.577000,169667,0.833000,False,0.000000,0.121000,-3.337000,1,81,0.069500,107.936000,4,0.580000
3,26,lee brice,rumor,26,25.000000,16.000000,2019-07-06,"November 3, 2017",Country,"Kyle jacobs, Ashley gorley, Lee brice",Rumor \nGirl you know I've known you forever\n...,NaN,36,0.749000,Lee Brice,0.655000,198440,0.560000,False,0.000000,0.115000,-6.857000,1,79,0.048600,140.975000,4,0.599000
4,32,panic! at the disco,high hopes,32,4.000000,47.000000,2019-07-06,"May 23, 2018","Adult Alternative,Rock,Power Pop,Pop-Rock,Alte...","Sam hollander, Cook classics, Tayla parx, Jake...","High Hopes \nHigh, high hopes\nHad to have hig...",NaN,81,0.193000,Pray for the Wicked,0.579000,190947,0.904000,False,0.000000,0.064000,-2.729000,1,87,0.061800,82.014000,4,0.681000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4768,97208,mark wills,wish you were here,81,34.000000,20.000000,1999-07-12,NaN,Country,"Debbie moore, Bill anderson, Skip ewing",Wish You Were Here \nThey kissed goodbye at th...,NaN,30,0.346000,Live At Billy Bob's Texas,0.533000,256993,0.329000,False,0.000000,0.391000,-14.857000,1,11,0.031000,136.094000,4,0.272000
4769,97208,mark wills,wish you were here,81,34.000000,20.000000,1999-07-12,NaN,Country,"Debbie moore, Bill anderson, Skip ewing",Wish You Were Here \nThey kissed goodbye at th...,NaN,90,0.190000,Wish You Were Here,0.513000,241960,0.398000,False,0.000000,0.117000,-10.234000,1,41,0.028500,138.045000,4,0.256000
4770,97212,clay walker,she's always right,85,74.000000,6.000000,1999-07-12,NaN,Pop,NaN,She's Always Right \nShe's Perfect for a cotto...,NaN,56,0.425000,"Live, Laugh, Love",0.459000,201867,0.539000,False,0.000000,0.143000,-10.020000,1,0,0.031500,142.685000,4,0.480000
4771,97222,collective soul,heavy,96,73.000000,20.000000,1999-07-12,NaN,"Hockey,Gaming,Soundtrack,Rock",Collective soul,Heavy \nComplicate this world you wrapped for ...,NaN,59,0.000877,Dosage,0.547000,173267,0.902000,False,0.000563,0.083900,-5.560000,1,50,0.037100,106.923000,4,0.548000


In [203]:
df_01["Features"].notna().value_counts()

False    4057
True      716
Name: Features, dtype: int64

# SVC MODEL

### Scaling data

In [204]:
scaled_data_arr = StandardScaler().fit_transform(
    df_01[[
        "Danceability",
        "Acousticness",
        "Energy",
        "Instrumentalness",
        "Liveness",
        "Duration",
        "Popularity",
        "Loudness",
        "Valence",
        "Speechiness"
    ]]
)

attributes_scaled_df= pd.DataFrame(scaled_data_arr, columns=[[
        "Danceability",
        "Acousticness",
        "Energy",
        "Instrumentalness",
        "Liveness",
        "Duration",
        "Popularity",
        "Loudness",
        "Valence",
        "Speechiness"
    ]])
attributes_scaled_df

,Danceability,Acousticness,Energy,Instrumentalness,Liveness,Duration,Popularity,Loudness,Valence,Speechiness
0,1.640820,-0.598410,0.178568,-0.138813,-0.546183,-1.208657,1.886360,0.356189,1.926516,-0.322514
1,1.143751,-0.756747,-0.186898,-0.138813,-0.814558,-1.422328,2.253012,0.122084,0.857317,-0.358839
2,-0.214437,-0.735848,0.752871,-0.138813,-0.451014,-1.459749,1.886360,1.089042,0.255331,-0.211464
3,0.331639,2.748893,-0.830813,-0.138813,-0.489081,-0.823774,1.794697,-0.403806,0.340687,-0.428376
4,-0.200435,0.113893,1.164745,-0.138813,-0.812655,-0.989393,2.161349,1.346898,0.709067,-0.291379
...,...,...,...,...,...,...,...,...,...,...
4768,-0.522479,0.838992,-2.170854,-0.138813,1.262023,0.470433,-1.321844,-3.796645,-1.128339,-0.611038
4769,-0.662499,0.099675,-1.770582,-0.138813,-0.476392,0.138156,0.053100,-1.836008,-1.200218,-0.636984
4770,-1.040551,1.213389,-0.952635,-0.138813,-0.311433,-0.748027,-1.825991,-1.745250,-0.193913,-0.605849
4771,-0.424466,-0.796618,1.153143,-0.131389,-0.686398,-1.380177,0.465584,0.146258,0.111573,-0.547729


In [205]:
attributes_scaled_df.corr()

,Danceability,Acousticness,Energy,Instrumentalness,Liveness,Duration,Popularity,Loudness,Valence,Speechiness
Danceability,1.000000,-0.117193,-0.012595,-0.033274,-0.112952,-0.060864,0.062896,0.031775,0.432008,0.210386
Acousticness,-0.117193,1.000000,-0.572402,0.005391,-0.048894,-0.007098,-0.068255,-0.452885,-0.178373,-0.050595
Energy,-0.012595,-0.572402,1.000000,-0.059236,0.138579,-0.107412,0.015134,0.722571,0.394901,0.035573
Instrumentalness,-0.033274,0.005391,-0.059236,1.000000,0.008549,0.041159,-0.026209,-0.129241,-0.061863,-0.062063
Liveness,-0.112952,-0.048894,0.138579,0.008549,1.000000,0.053116,-0.062582,0.040298,0.005788,0.136006
Duration,-0.060864,-0.007098,-0.107412,0.041159,0.053116,1.000000,0.029971,-0.080970,-0.228600,0.078036
Popularity,0.062896,-0.068255,0.015134,-0.026209,-0.062582,0.029971,1.000000,0.039605,-0.005686,0.056787
Loudness,0.031775,-0.452885,0.722571,-0.129241,0.040298,-0.080970,0.039605,1.000000,0.257349,-0.037841
Valence,0.432008,-0.178373,0.394901,-0.061863,0.005788,-0.228600,-0.005686,0.257349,1.000000,0.080014
Speechiness,0.210386,-0.050595,0.035573,-0.062063,0.136006,0.078036,0.056787,-0.037841,0.080014,1.000000


### Popularity to binary

In [206]:
pd.set_option('display.float_format', '{:.6f}'.format)
attributes_scaled_df["Popularity"].describe()
x = 0.831846
attributes_scaled_df['Popularity'] = np.where((attributes_scaled_df['Popularity'] > x) & attributes_scaled_df['Popularity']
                                            .notna(), 1, 0)

In [207]:
# Checking data frame
attributes_scaled_df
# attributes_scaled_df["Popularity"].value_counts()

,Danceability,Acousticness,Energy,Instrumentalness,Liveness,Duration,Popularity,Loudness,Valence,Speechiness
0,1.640820,-0.598410,0.178568,-0.138813,-0.546183,-1.208657,1,0.356189,1.926516,-0.322514
1,1.143751,-0.756747,-0.186898,-0.138813,-0.814558,-1.422328,1,0.122084,0.857317,-0.358839
2,-0.214437,-0.735848,0.752871,-0.138813,-0.451014,-1.459749,1,1.089042,0.255331,-0.211464
3,0.331639,2.748893,-0.830813,-0.138813,-0.489081,-0.823774,1,-0.403806,0.340687,-0.428376
4,-0.200435,0.113893,1.164745,-0.138813,-0.812655,-0.989393,1,1.346898,0.709067,-0.291379
...,...,...,...,...,...,...,...,...,...,...
4768,-0.522479,0.838992,-2.170854,-0.138813,1.262023,0.470433,0,-3.796645,-1.128339,-0.611038
4769,-0.662499,0.099675,-1.770582,-0.138813,-0.476392,0.138156,0,-1.836008,-1.200218,-0.636984
4770,-1.040551,1.213389,-0.952635,-0.138813,-0.311433,-0.748027,0,-1.745250,-0.193913,-0.605849
4771,-0.424466,-0.796618,1.153143,-0.131389,-0.686398,-1.380177,0,0.146258,0.111573,-0.547729


### Preparing model

In [208]:
# preparing the data for the model
popularity = attributes_scaled_df["Popularity"]
target_popularity=["Popular","Not popular"]
X = attributes_scaled_df.drop(columns=["Popularity"])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_21740\2778729577.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = attributes_scaled_df.drop(columns=["Popularity"])


In [209]:
X

,Danceability,Acousticness,Energy,Instrumentalness,Liveness,Duration,Loudness,Valence,Speechiness
0,1.640820,-0.598410,0.178568,-0.138813,-0.546183,-1.208657,0.356189,1.926516,-0.322514
1,1.143751,-0.756747,-0.186898,-0.138813,-0.814558,-1.422328,0.122084,0.857317,-0.358839
2,-0.214437,-0.735848,0.752871,-0.138813,-0.451014,-1.459749,1.089042,0.255331,-0.211464
3,0.331639,2.748893,-0.830813,-0.138813,-0.489081,-0.823774,-0.403806,0.340687,-0.428376
4,-0.200435,0.113893,1.164745,-0.138813,-0.812655,-0.989393,1.346898,0.709067,-0.291379
...,...,...,...,...,...,...,...,...,...
4768,-0.522479,0.838992,-2.170854,-0.138813,1.262023,0.470433,-3.796645,-1.128339,-0.611038
4769,-0.662499,0.099675,-1.770582,-0.138813,-0.476392,0.138156,-1.836008,-1.200218,-0.636984
4770,-1.040551,1.213389,-0.952635,-0.138813,-0.311433,-0.748027,-1.745250,-0.193913,-0.605849
4771,-0.424466,-0.796618,1.153143,-0.131389,-0.686398,-1.380177,0.146258,0.111573,-0.547729


In [210]:
popularity.value_counts()

(Popularity,)
0                3554
1                1219
dtype: int64

In [211]:
# Data split
X_train, X_test, y_train, y_test = train_test_split(X, popularity, random_state=42, stratify=popularity)

In [212]:
y_train

,Popularity
2568,0
3826,0
1410,0
445,1
1795,0
...,...
2972,0
2726,1
2244,0
93,1


### Model with standarized data


In [213]:
model = SVC(kernel="linear", random_state=42)
model.fit(X_train, y_train)

c:\Users\Usuario\anaconda3\envs\dev\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear', random_state=42)

### Results 1

In [214]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

Training Data Score: 0.744621402626432
Testing Data Score: 0.7445561139028476


In [215]:
# Calculate the classification report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_popularity))

              precision    recall  f1-score   support

     Popular       0.74      1.00      0.85       889
 Not popular       0.00      0.00      0.00       305

    accuracy                           0.74      1194
   macro avg       0.37      0.50      0.43      1194
weighted avg       0.55      0.74      0.64      1194



c:\Users\Usuario\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Usuario\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Usuario\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Model with resampled data

In [216]:
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_res, y_res = ros.fit_resample(X_train, y_train)

In [217]:
model2 = SVC(kernel="linear", random_state=42)
# Fit the model using training data
model2.fit(X_train, y_train)

# Fit the model using the resampled training data
model2.fit(X_res, y_res)

# Make a prediction using the testing data
predictions2 = model2.predict(X_test)

c:\Users\Usuario\anaconda3\envs\dev\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Usuario\anaconda3\envs\dev\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Results 2

In [218]:
print(f"Training Data Score: {model2.score(X_train, y_train)}")
print(f"Testing Data Score: {model2.score(X_test, y_test)}")

Training Data Score: 0.6387259010896899
Testing Data Score: 0.6139028475711893


In [219]:
print(classification_report(y_test, predictions2,
                            target_names=target_popularity))

              precision    recall  f1-score   support

     Popular       0.75      0.72      0.73       889
 Not popular       0.28      0.31      0.29       305

    accuracy                           0.61      1194
   macro avg       0.51      0.52      0.51      1194
weighted avg       0.63      0.61      0.62      1194



### Model using features

In [220]:
# Replacing feature with true and false values.

df_01['Features'] = np.where(df_01['Features'].notna(), 1, 0)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_21740\1647659694.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_01['Features'] = np.where(df_01['Features'].notna(), 1, 0)


In [221]:
df_01["Features"].value_counts()

0    4057
1     716
Name: Features, dtype: int64

In [222]:
df_01

,Unnamed: 0,Artist,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features,X,Acousticness,Album,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,TimeSignature,Valence
0,6,jonas brothers,sucker,6,1.000000,17.000000,2019-07-06,"March 1, 2019","Alternative Pop,Boy Band,Teen Pop,Pop-Rock,Pop","Ryan tedder, Louis bell, Frank dukes, Nick jon...",Sucker \nWe go together\nBetter than birds of ...,0,12,0.042700,Happiness Begins,0.842000,181027,0.734000,False,0.000000,0.106000,-5.065000,0,81,0.058800,137.958000,4,0.952000
1,13,taylor swift,you need to calm down,13,2.000000,2.000000,2019-07-06,"June 14, 2019","Synth-Pop,LGBTQ+,Pop","Joel little, Taylor swift",You Need To Calm Down \nYou are somebody that ...,0,113,0.009290,Lover,0.771000,171360,0.671000,False,0.000000,0.063700,-5.617000,1,89,0.055300,85.026000,4,0.714000
2,24,panic! at the disco,"hey look ma, i made it",24,24.000000,11.000000,2019-07-06,"June 22, 2018","Pop-Rock,Jazz Fusion,Alternative,Alternative P...","Jake sinclair, Michael angelakos, Dillon franc...","Hey Look Ma, I Made It \nAll my life, been hus...",0,80,0.013700,Pray for the Wicked,0.577000,169667,0.833000,False,0.000000,0.121000,-3.337000,1,81,0.069500,107.936000,4,0.580000
3,26,lee brice,rumor,26,25.000000,16.000000,2019-07-06,"November 3, 2017",Country,"Kyle jacobs, Ashley gorley, Lee brice",Rumor \nGirl you know I've known you forever\n...,0,36,0.749000,Lee Brice,0.655000,198440,0.560000,False,0.000000,0.115000,-6.857000,1,79,0.048600,140.975000,4,0.599000
4,32,panic! at the disco,high hopes,32,4.000000,47.000000,2019-07-06,"May 23, 2018","Adult Alternative,Rock,Power Pop,Pop-Rock,Alte...","Sam hollander, Cook classics, Tayla parx, Jake...","High Hopes \nHigh, high hopes\nHad to have hig...",0,81,0.193000,Pray for the Wicked,0.579000,190947,0.904000,False,0.000000,0.064000,-2.729000,1,87,0.061800,82.014000,4,0.681000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4768,97208,mark wills,wish you were here,81,34.000000,20.000000,1999-07-12,NaN,Country,"Debbie moore, Bill anderson, Skip ewing",Wish You Were Here \nThey kissed goodbye at th...,0,30,0.346000,Live At Billy Bob's Texas,0.533000,256993,0.329000,False,0.000000,0.391000,-14.857000,1,11,0.031000,136.094000,4,0.272000
4769,97208,mark wills,wish you were here,81,34.000000,20.000000,1999-07-12,NaN,Country,"Debbie moore, Bill anderson, Skip ewing",Wish You Were Here \nThey kissed goodbye at th...,0,90,0.190000,Wish You Were Here,0.513000,241960,0.398000,False,0.000000,0.117000,-10.234000,1,41,0.028500,138.045000,4,0.256000
4770,97212,clay walker,she's always right,85,74.000000,6.000000,1999-07-12,NaN,Pop,NaN,She's Always Right \nShe's Perfect for a cotto...,0,56,0.425000,"Live, Laugh, Love",0.459000,201867,0.539000,False,0.000000,0.143000,-10.020000,1,0,0.031500,142.685000,4,0.480000
4771,97222,collective soul,heavy,96,73.000000,20.000000,1999-07-12,NaN,"Hockey,Gaming,Soundtrack,Rock",Collective soul,Heavy \nComplicate this world you wrapped for ...,0,59,0.000877,Dosage,0.547000,173267,0.902000,False,0.000563,0.083900,-5.560000,1,50,0.037100,106.923000,4,0.548000


In [223]:
X["Features"] = df_01["Features"]

In [224]:
X_train, X_test, y_train, y_test = train_test_split(X, popularity, random_state=42, stratify=popularity)

ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_res, y_res = ros.fit_resample(X_train, y_train)

model3 = SVC(kernel="linear", random_state=42)
# Fit the model using training data
model3.fit(X_train, y_train)

# Fit the model using the resampled training data
model3.fit(X_res, y_res)

# Make a prediction using the testing data
predictions3 = model3.predict(X_test)

c:\Users\Usuario\anaconda3\envs\dev\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Usuario\anaconda3\envs\dev\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [225]:
y_res.value_counts()

(Popularity,)
0                2665
1                2665
dtype: int64

### Results 3

In [226]:
print(f"Training Data Score: {model3.score(X_train, y_train)}")
print(f"Testing Data Score: {model3.score(X_test, y_test)}")

Training Data Score: 0.6306230790723666
Testing Data Score: 0.6130653266331658


In [227]:
print(classification_report(y_test, predictions3,
                            target_names=target_popularity))

              precision    recall  f1-score   support

     Popular       0.76      0.71      0.73       889
 Not popular       0.28      0.33      0.30       305

    accuracy                           0.61      1194
   macro avg       0.52      0.52      0.52      1194
weighted avg       0.63      0.61      0.62      1194



# DECISION TREES MODEL

In [228]:
df_01.drop(columns=["X"], inplace=True)
df_DT=df_01.dropna()
df_DT


C:\Users\Usuario\AppData\Local\Temp\ipykernel_21740\1129947268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_01.drop(columns=["X"], inplace=True)


,Unnamed: 0,Artist,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features,Acousticness,Album,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,TimeSignature,Valence
0,6,jonas brothers,sucker,6,1.000000,17.000000,2019-07-06,"March 1, 2019","Alternative Pop,Boy Band,Teen Pop,Pop-Rock,Pop","Ryan tedder, Louis bell, Frank dukes, Nick jon...",Sucker \nWe go together\nBetter than birds of ...,0,0.042700,Happiness Begins,0.842000,181027,0.734000,False,0.000000,0.106000,-5.065000,0,81,0.058800,137.958000,4,0.952000
1,13,taylor swift,you need to calm down,13,2.000000,2.000000,2019-07-06,"June 14, 2019","Synth-Pop,LGBTQ+,Pop","Joel little, Taylor swift",You Need To Calm Down \nYou are somebody that ...,0,0.009290,Lover,0.771000,171360,0.671000,False,0.000000,0.063700,-5.617000,1,89,0.055300,85.026000,4,0.714000
2,24,panic! at the disco,"hey look ma, i made it",24,24.000000,11.000000,2019-07-06,"June 22, 2018","Pop-Rock,Jazz Fusion,Alternative,Alternative P...","Jake sinclair, Michael angelakos, Dillon franc...","Hey Look Ma, I Made It \nAll my life, been hus...",0,0.013700,Pray for the Wicked,0.577000,169667,0.833000,False,0.000000,0.121000,-3.337000,1,81,0.069500,107.936000,4,0.580000
3,26,lee brice,rumor,26,25.000000,16.000000,2019-07-06,"November 3, 2017",Country,"Kyle jacobs, Ashley gorley, Lee brice",Rumor \nGirl you know I've known you forever\n...,0,0.749000,Lee Brice,0.655000,198440,0.560000,False,0.000000,0.115000,-6.857000,1,79,0.048600,140.975000,4,0.599000
4,32,panic! at the disco,high hopes,32,4.000000,47.000000,2019-07-06,"May 23, 2018","Adult Alternative,Rock,Power Pop,Pop-Rock,Alte...","Sam hollander, Cook classics, Tayla parx, Jake...","High Hopes \nHigh, high hopes\nHad to have hig...",0,0.193000,Pray for the Wicked,0.579000,190947,0.904000,False,0.000000,0.064000,-2.729000,1,87,0.061800,82.014000,4,0.681000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4577,93699,madonna,american pie,88,29.000000,9.000000,2000-04-11,"March 3, 2000","Soundtrack,Cover,Pop",Don mclean,"American Pie \nA long, long time ago\nI can st...",0,0.316000,Music,0.634000,271853,0.674000,False,0.000000,0.139000,-9.750000,0,60,0.037000,124.001000,4,0.523000
4597,94076,amel larrieux,get up,97,97.000000,3.000000,2000-03-14,"February 15, 2000","Soul,Neo Soul,R&;B","Laru larrieux, Amel larrieux","Get Up \nMmm la dat dat, da dat dat\nDa da da ...",0,0.310000,Infinite Possibilities,0.631000,244867,0.494000,False,0.000018,0.064000,-8.335000,0,43,0.101000,88.559000,4,0.542000
4598,94077,tracie spencer,still in my heart,98,88.000000,3.000000,2000-03-14,"March 14, 2000",R&;B,"Ivan matias, Andrea martin, Soulshock and karlin","Still in My Heart \nOh, I just want you to kno...",0,0.307000,Tracie,0.674000,257240,0.529000,False,0.000002,0.072500,-6.188000,1,30,0.025000,89.913000,4,0.601000
4616,94263,will smith,freakin' it,99,99.000000,4.000000,2000-02-28,"March 22, 2000",Rap,"Nile rodgers, Bernard edwards, Marilyn mcleod,...","Freaking It \nUh, uh\nI'm bout to freak this\n...",0,0.198000,Willennium,0.873000,239293,0.833000,False,0.000000,0.237000,-5.511000,1,40,0.189000,112.001000,4,0.927000


In [229]:
df_DT = df_DT.drop(columns= ["Name", "Unnamed: 0", "Album", 
                             "Artist", "Week", "Date", "Genre",
                             "Writing.Credits","Lyrics", "Weekly.rank",
                            "Peak.position", "Explicit", "Mode",
                            "Tempo","TimeSignature"])
df_DT["Popularity"].describe()
df_DT = pd.get_dummies(df_DT)
df_DT['Is Popular'] = np.where((df_DT['Popularity'] > 50) & df_DT['Popularity']
                                            .notna(), 1, 0)
df_DT = df_DT.drop(columns= ["Popularity"])
df_DT

,Weeks.on.chart,Features,Acousticness,Danceability,Duration,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Valence,Is Popular
0,17.000000,0,0.042700,0.842000,181027,0.734000,0.000000,0.106000,-5.065000,0.058800,0.952000,1
1,2.000000,0,0.009290,0.771000,171360,0.671000,0.000000,0.063700,-5.617000,0.055300,0.714000,1
2,11.000000,0,0.013700,0.577000,169667,0.833000,0.000000,0.121000,-3.337000,0.069500,0.580000,1
3,16.000000,0,0.749000,0.655000,198440,0.560000,0.000000,0.115000,-6.857000,0.048600,0.599000,1
4,47.000000,0,0.193000,0.579000,190947,0.904000,0.000000,0.064000,-2.729000,0.061800,0.681000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4577,9.000000,0,0.316000,0.634000,271853,0.674000,0.000000,0.139000,-9.750000,0.037000,0.523000,1
4597,3.000000,0,0.310000,0.631000,244867,0.494000,0.000018,0.064000,-8.335000,0.101000,0.542000,0
4598,3.000000,0,0.307000,0.674000,257240,0.529000,0.000002,0.072500,-6.188000,0.025000,0.601000,0
4616,4.000000,0,0.198000,0.873000,239293,0.833000,0.000000,0.237000,-5.511000,0.189000,0.927000,0


In [230]:
# Define features set
X = df_DT.copy()
X.drop('Is Popular', axis=1, inplace=True)
X.head()

,Weeks.on.chart,Features,Acousticness,Danceability,Duration,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Valence
0,17.000000,0,0.042700,0.842000,181027,0.734000,0.000000,0.106000,-5.065000,0.058800,0.952000
1,2.000000,0,0.009290,0.771000,171360,0.671000,0.000000,0.063700,-5.617000,0.055300,0.714000
2,11.000000,0,0.013700,0.577000,169667,0.833000,0.000000,0.121000,-3.337000,0.069500,0.580000
3,16.000000,0,0.749000,0.655000,198440,0.560000,0.000000,0.115000,-6.857000,0.048600,0.599000
4,47.000000,0,0.193000,0.579000,190947,0.904000,0.000000,0.064000,-2.729000,0.061800,0.681000


In [231]:
y = df_DT['Is Popular'].values
y


array([1, 1, 1, ..., 0, 0, 0])

In [232]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [233]:
# Creating StandardScaler instance
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [234]:
#Creating the Model
tree_model = tree.DecisionTreeClassifier()
tree_model = tree_model.fit(X_train_scaled, y_train)
predictions_tree = tree_model.predict(X_test_scaled)

In [235]:
#Evaluating the model
cm = confusion_matrix(y_test, predictions_tree)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions_tree)

print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions_tree))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,304,165
Actual 1,208,156


Accuracy Score : 0.5522208883553421
Classification Report
              precision    recall  f1-score   support

           0       0.59      0.65      0.62       469
           1       0.49      0.43      0.46       364

    accuracy                           0.55       833
   macro avg       0.54      0.54      0.54       833
weighted avg       0.55      0.55      0.55       833

